In [53]:
import glob
from skimage.measure import marching_cubes_lewiner, marching_cubes
import nibabel
import os
import trimesh
import numpy as np
from natsort import natsorted
import re
import pymeshlab
sorted = natsorted
from numba import njit
import cc3d
from scipy.ndimage import gaussian_filter
import matplotlib.pyplot as plt
from IPython.core.debugger import set_trace
from shutil import rmtree

from tqdm import tqdm_notebook

%config Completer.use_jedi = False

BRAIN_FOLDER = '../fcd_newdataset/brain_folder/'
MASK_FOLDER = '../fcd_newdataset/mask_folder/'


SAVE_FOLDER_INNER = '../fcd_newdataset_meshes/inner'
SAVE_FOLDER_OUTER = '../fcd_newdataset_meshes/outer/'
SAVE_FOLDER_INNER_OUTER = '../fcd_newdataset_meshes/inner_and_outer/'

REBUILT = False
if REBUILT:
    rmtree(SAVE_FOLDER_INNER)
    rmtree(SAVE_FOLDER_OUTER)
    rmtree(SAVE_FOLDER_INNER_OUTER)
    
for f in [SAVE_FOLDER_INNER, SAVE_FOLDER_OUTER, SAVE_FOLDER_INNER_OUTER]:
    os.makedirs(f, exist_ok=True)
    os.makedirs(os.path.join(f,'seg'), exist_ok=True)
    os.makedirs(os.path.join(f,'sseg'), exist_ok=True)

brain_mask_names = sorted(glob.glob(MASK_FOLDER + '*.nii.gz'))
labels = list(map(lambda x: re.findall('\d+\.\d+',x), brain_mask_names))

brain_img_names = {}
brain_mask_names = {}

for [label] in labels:
    
    brain_path = os.path.join(BRAIN_FOLDER, f'fcd_{label}.nii.gz')
    mask_path = os.path.join(MASK_FOLDER, f'mask_fcd_{label}.nii.gz')
    
    if os.path.isfile(brain_path):
        
        brain_img_names[label] = brain_path
        brain_mask_names[label] = mask_path

/hdd/ibulygin/miniconda3/envs/fcd_hpc2/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning: Config option `use_jedi` not recognized by `IPCompleter`.


# TODO

 - connect falling apart components into one
 - build full w\g-matter graphs

## Util functions

In [54]:
def count_edges(faces):
    """
        input: array of faces
        output: dict, keys = id of edge, value = faces that contains this edge
    """
    edge_dict = {}
    for face in faces:
        keys = [
            tuple(sorted((face[0], face[1]))), tuple(sorted((face[1], face[2]))), tuple(sorted((face[0], face[2])))
        ]
        for key in keys:
            edge_dict[key] = edge_dict.get(key, 0) + 1
    return edge_dict

def get_vertex_labels(verts, mask):
    """
        verts: list of 3d coordinates of vertices
        mask: 3d binary array of mask
        
        labels: binary labels for each vertice
    """
    vs = verts.astype('int32')
    labels = []
    for v_ in vs:
        a, b, c = v_
        labels += [mask[a, b, c]]
    labels = np.array(labels)
    return labels

def get_edge_labels(v_labels, e_dict):
    """
        v_labels: vertex_labels
        e_dict: edge_dict
        
        e_labels: binary labels for each edge
    """
    es = list(e_dict.keys())
    e_labels = []
    for i, (v1, v2) in enumerate(es):
        e_labels += [v_labels[v1] or v_labels[v2]]
    e_labels = np.array(e_labels)
    return e_labels

@njit
def morph_3d(mask, l=5):
    """
        3d dilation for masks
    """
    d, h, w = mask.shape
    mask_padded = np.zeros((d+2*l//2, h+2*l//2, w+2*l//2))
    mask_padded[l//2: -l//2, l//2: -l//2, l//2: -l//2] = mask.copy()
    for i in range(d):
        for j in range(h):
            for k in range(w):
                mask[i, j, k] = np.all(mask_padded[i: i + l, j: j + l, k: k + l])
                
def show_slices(brain_tensor, n_slices_show=5, mask_tensor=None):
    
    fig, axes = plt.subplots(ncols=3, nrows=n_slices_show, figsize=(15,n_slices_show*5))
    X_max, Y_max, Z_max = brain_tensor.shape
    for i in range(n_slices_show):

        x_slice_pos = (X_max//(n_slices_show+2))*(i+1)
        y_slice_pos = (Y_max//(n_slices_show+2))*(i+1)
        z_slice_pos = (Z_max//(n_slices_show+2))*(i+1)

        brain_tensor_x_slice = brain_tensor[x_slice_pos,:,:]
        brain_tensor_y_slice = brain_tensor[:,y_slice_pos,:]
        brain_tensor_z_slice = brain_tensor[:,:,z_slice_pos]

        axes[i,0].imshow(brain_tensor_x_slice, 'gray')
        axes[i,1].imshow(brain_tensor_y_slice, 'gray')
        axes[i,2].imshow(brain_tensor_z_slice, 'gray')
        
        if mask is not None:
            
            mask_tensor_x_slice = mask_tensor[x_slice_pos,:,:]
            mask_tensor_y_slice = mask_tensor[:,y_slice_pos,:]
            mask_tensor_z_slice = mask_tensor[:,:,z_slice_pos]

            axes[i,0].imshow(mask_tensor_x_slice, 'jet', interpolation='none', alpha=0.7)
            axes[i,1].imshow(mask_tensor_y_slice, 'jet', interpolation='none', alpha=0.7)
            axes[i,2].imshow(mask_tensor_z_slice, 'jet', interpolation='none', alpha=0.7)

    plt.show()

##  Extracting example

In [55]:
# label = '10.1'
# br_name = brain_img_names[label]

# brain = nibabel.load(br_name)
# mask = nibabel.load(brain_mask_names[label]).get_fdata() > 0 # why cast to bool? 
# brain_tensor_orig = brain.get_fdata() 

In [56]:
# plt.hist(brain_tensor_orig[brain_tensor_orig>0].flatten(), bins=100)
# plt.show()

In [57]:
# show_slices(brain_tensor_orig, n_slices_show=2, mask_tensor=brain_tensor_orig > white_matter_thresold)

In [58]:
# white_matter_thresold = 300

# brain_tensor_white = brain_tensor_orig > white_matter_thresold # for the inner
# l = 1
# t = 0.4
# brain_tensor_white = gaussian_filter(brain_tensor_white.astype('float'), sigma=l)
# brain_tensor_white = brain_tensor_white > t # gray matter vs white matter? 
# labels_in = brain_tensor_white # where is a white matter

In [59]:
# show_slices(brain_tensor_orig, n_slices_show=2, mask_tensor=labels_in)

In [60]:
# labels_out = cc3d.connected_components(labels_in, connectivity=6)
# u, c = np.unique(labels_out, return_counts=True)
# max_connected_label = np.argmax(c[1:]) + 1
# brain_tensor_white = labels_out == max_connected_label

In [61]:
# show_slices(brain_tensor_orig, n_slices_show=2, mask_tensor=brain_tensor_white)

In [62]:
# verts, faces, normals, values = marching_cubes(brain_tensor_white, 
#                                                step_size=1, # 6
#                                                allow_degenerate=False, 
#                                                gradient_direction='ascent')

In [63]:
# mesh_n = trimesh.base.Trimesh(vertices = verts, faces = faces, vertex_normals=normals, process = True)
# mesh_n.export("example_mesh.obj")

In [64]:
# with open('example_mesh.obj') as file:
#     data = file.read()

In [65]:
# data = data.split('\n')[1:]
# types = []
# for obj in tqdm_notebook(data):
#     s = obj[:2].strip()
#     if s not in types:
#         types.append(s)

# Extracting all meshes

In [ ]:
for mesh_type,SAVE_FOLDER in {'inner':SAVE_FOLDER_INNER, 
                              'outer':SAVE_FOLDER_OUTER}.items():
    
    for i, (label,br_name) in tqdm_notebook(enumerate(brain_img_names.items())):

        brain_tensor = nibabel.load(br_name).get_fdata()  > 0 
        mask_tensor = nibabel.load(brain_mask_names[label]).get_fdata() > 0
        
        if mesh_type == 'inner':
            brain_tensor = brain.get_fdata() > 300
            l = 1
            t = 0.4
            brain_tensor = gaussian_filter(brain_tensor.astype('float'), sigma=l)
            brain_tensor = brain_tensor > 0.4 # gray matter vs white matter? 
            labels_in = brain_tensor

            labels_out = cc3d.connected_components(labels_in, connectivity=6)
            u, c = np.unique(labels_out, return_counts=True)
            max_connected_label = np.argmax(c[1:]) + 1
            brain_tensor = labels_out == max_connected_label

        else:
            brain_tensor = brain.get_fdata() > 0

        verts, faces, normals, values = marching_cubes(brain_tensor, 
                                                       step_size=1, 
                                                       allow_degenerate=False, 
                                                       gradient_direction = 'ascent')
        
        v_labels = get_vertex_labels(verts, mask)
        print('vertex labels mean', v_labels.mean())
        
        e_dict = count_edges(faces)
        np_e_dict = np.array(list(e_dict.values()))
        e_labels = get_edge_labels(v_labels, e_dict)
        print('edge labels mean', v_labels.mean())

        print('number of manifold edges', (np_e_dict == 2).sum())
        print('number of non-manifold edges', (np_e_dict != 2).sum())

        mesh_n = trimesh.base.Trimesh(vertices = verts, faces = faces, process = False)
        mesh_n.export(os.path.join(SAVE_FOLDER, f"{label}.obj")) # 'v', 'vn', 'f'
 
        seg = e_labels + 1
        sseg = np.zeros((len(e_dict), 2), dtype=np.int32)
        sseg[np.arange(seg.size), seg-1] = 1

        with open(os.path.join(SAVE_FOLDER, f"{label}.obj"), 'a') as f:
            for j, e in enumerate(e_dict):
                f.write(f'\ne {e[0]} {e[1]} {seg[j]}') # why add edges?
        
        # remove first line with trimesh github link
        with open(os.path.join(SAVE_FOLDER, f"{label}.obj"), 'r') as fin:
            data = fin.read().splitlines(True)
        with open(os.path.join(SAVE_FOLDER, f"{label}.obj"), 'w') as fout:
            fout.writelines(data[1:]) 
        
        # save labels
        np.savetxt(os.path.join(SAVE_FOLDER, f'seg/{label}.eseg'), seg)
        np.savetxt(os.path.join(SAVE_FOLDER, f'sseg/{label}.seseg'), sseg)

/hdd/ibulygin/miniconda3/envs/fcd_hpc2/lib/python3.6/site-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


0it [00:00, ?it/s]

vertex labels mean 0.0011575936553691038
edge labels mean 0.0011575936553691038
number of manifold edges 4721859
number of non-manifold edges 0
vertex labels mean 0.0011575936553691038
edge labels mean 0.0011575936553691038
number of manifold edges 4721859
number of non-manifold edges 0
vertex labels mean 0.0011575936553691038
edge labels mean 0.0011575936553691038
number of manifold edges 4721859
number of non-manifold edges 0
vertex labels mean 0.0011575936553691038
edge labels mean 0.0011575936553691038
number of manifold edges 4721859
number of non-manifold edges 0
vertex labels mean 0.0011575936553691038
edge labels mean 0.0011575936553691038
number of manifold edges 4721859
number of non-manifold edges 0
vertex labels mean 0.0011575936553691038
edge labels mean 0.0011575936553691038
number of manifold edges 4721859
number of non-manifold edges 0
vertex labels mean 0.0011575936553691038
edge labels mean 0.0011575936553691038
number of manifold edges 4721859
number of non-manifold 

##  Combining inner and outter meshes

In [68]:
def count_vertices(data):
    """
        return count of vertices, faces and edges
    """
    for j in range(len(data)):
        if data[j][0] != 'v':
            a = j
            break
    for j in range(a, len(data)):
        if data[j][0] != 'f':
            b = j
            break
    return a, b - a, len(data) - b

def shift_line(line, n):
    """
        line: line from text description of .obj file. it starts with 'v', 'e' or 'f'
        n: shift indexing by this number
        
        return: shifted line
    """
    ls = line.split(' ')
    joins = [ls[0], str(int(ls[1]) + n), str(int(ls[2]) + n)]
    try:
        if ls[0] == 'e':
            joins.append(str(int(ls[3])))
        else:
            joins.append(str(int(ls[3]) + n))
    except:
        print(line)
        print(ls)
        raise ValueError
    return " ".join(joins) + '\n'

In [71]:
label = '0.1'
with open(os.path.join(SAVE_FOLDER_INNER, f"{label}.obj"), 'r') as fin:
    data_inner = fin.read().splitlines(True)
        
with open(os.path.join(SAVE_FOLDER_OUTER, f"{label}.obj"), 'r') as fin:
    data_outter = fin.read().splitlines(True)

In [83]:
len(data_outter)

3130848

In [78]:
list(filter(lambda l: l[0]=='f', data_outter))

['f 1 2 3\n',
 'f 1 4 5\n',
 'f 3 4 1\n',
 'f 5 6 7\n',
 'f 4 6 5\n',
 'f 7 8 9\n',
 'f 6 8 7\n',
 'f 9 10 11\n',
 'f 8 10 9\n',
 'f 11 12 13\n',
 'f 10 12 11\n',
 'f 12 14 13\n',
 'f 15 16 17\n',
 'f 2 15 17\n',
 'f 2 1 15\n',
 'f 1 18 15\n',
 'f 5 19 1\n',
 'f 19 18 1\n',
 'f 7 20 5\n',
 'f 20 19 5\n',
 'f 9 21 7\n',
 'f 21 20 7\n',
 'f 11 22 9\n',
 'f 22 21 9\n',
 'f 13 23 11\n',
 'f 23 22 11\n',
 'f 24 13 14\n',
 'f 24 25 13\n',
 'f 25 23 13\n',
 'f 25 26 27\n',
 'f 24 26 25\n',
 'f 26 28 27\n',
 'f 29 30 31\n',
 'f 29 32 33\n',
 'f 31 32 29\n',
 'f 16 33 32\n',
 'f 16 15 33\n',
 'f 15 34 33\n',
 'f 18 35 15\n',
 'f 35 34 15\n',
 'f 19 36 18\n',
 'f 36 35 18\n',
 'f 20 37 19\n',
 'f 37 36 19\n',
 'f 21 38 20\n',
 'f 38 37 20\n',
 'f 22 39 21\n',
 'f 39 38 21\n',
 'f 23 40 22\n',
 'f 40 39 22\n',
 'f 25 41 23\n',
 'f 41 40 23\n',
 'f 27 42 25\n',
 'f 42 41 25\n',
 'f 43 27 28\n',
 'f 43 44 27\n',
 'f 44 42 27\n',
 'f 43 45 44\n',
 'f 46 47 48\n',
 'f 46 49 50\n',
 'f 48 49 46\n',
 '

In [79]:
list(filter(lambda l: l[0]=='e', data_outter))

['e 0 1 1\n',
 'e 1 2 1\n',
 'e 0 2 1\n',
 'e 0 3 1\n',
 'e 3 4 1\n',
 'e 0 4 1\n',
 'e 2 3 1\n',
 'e 4 5 1\n',
 'e 5 6 1\n',
 'e 4 6 1\n',
 'e 3 5 1\n',
 'e 6 7 1\n',
 'e 7 8 1\n',
 'e 6 8 1\n',
 'e 5 7 1\n',
 'e 8 9 1\n',
 'e 9 10 1\n',
 'e 8 10 1\n',
 'e 7 9 1\n',
 'e 10 11 1\n',
 'e 11 12 1\n',
 'e 10 12 1\n',
 'e 9 11 1\n',
 'e 11 13 1\n',
 'e 12 13 1\n',
 'e 14 15 1\n',
 'e 15 16 1\n',
 'e 14 16 1\n',
 'e 1 14 1\n',
 'e 1 16 1\n',
 'e 0 14 1\n',
 'e 0 17 1\n',
 'e 14 17 1\n',
 'e 4 18 1\n',
 'e 0 18 1\n',
 'e 17 18 1\n',
 'e 6 19 1\n',
 'e 4 19 1\n',
 'e 18 19 1\n',
 'e 8 20 1\n',
 'e 6 20 1\n',
 'e 19 20 1\n',
 'e 10 21 1\n',
 'e 8 21 1\n',
 'e 20 21 1\n',
 'e 12 22 1\n',
 'e 10 22 1\n',
 'e 21 22 1\n',
 'e 12 23 1\n',
 'e 13 23 1\n',
 'e 23 24 1\n',
 'e 12 24 1\n',
 'e 22 24 1\n',
 'e 24 25 1\n',
 'e 25 26 1\n',
 'e 24 26 1\n',
 'e 23 25 1\n',
 'e 25 27 1\n',
 'e 26 27 1\n',
 'e 28 29 1\n',
 'e 29 30 1\n',
 'e 28 30 1\n',
 'e 28 31 1\n',
 'e 31 32 1\n',
 'e 28 32 1\n',
 'e 30 3

In [84]:
with open(os.path.join(SAVE_FOLDER_INNER, f"{label}.obj"), 'r') as fin:
    data_inner_text = fin.read()

In [93]:
np.unique(re.findall('[a-z]+',data_inner_text.replace('\n',' ')))

array(['e', 'f', 'v'], dtype='<U1')

In [ ]:
for label in tqdm_notebook(brain_img_names.keys()):
    
    with open(os.path.join(SAVE_FOLDER_INNER, f"{label}.obj"), 'r') as fin:
        data_inner = fin.read().splitlines(True)
        
    with open(os.path.join(SAVE_FOLDER_OUTER, f"{label}.obj"), 'r') as fin:
        data_outter = fin.read().splitlines(True)
        
    a1, b1, c1 = count_vertices(data_inner) # vertices, faces and edges
    a2, b2, c2 = count_vertices(data_outter) 

    data = []
    
    # vertices
    data += data_inner[:a1] # inner vertices
    data += data_outter[:a2] # inner + outer vertices
    
    # faces
    data += data_inner[a1: a1 + b1] # inner + outer vertices + inner faces
    data += list(map(lambda x: shift_line(x, a1), data_outter[a2: a2 + b2])) # (inner + outer vertices) + (inner + outer faces)
    
    # edges
    data += data_inner[a1 + b1:]
    data += list(map(lambda x: shift_line(x, a1), data_outter[a2 + b2:]))

    with open(SAVE_FOLDER_INNER_OUTER + f"{label}.obj", 'w') as fout:
        fout.writelines(data)
    
    seg_inner = np.loadtxt(os.path.join(SAVE_FOLDER_INNER, 'seg/' + f"{label}.eseg"))
    sseg_inner = np.loadtxt(os.path.join(SAVE_FOLDER_INNER, 'sseg/' + f"{label}.seseg"))    
    
    seg_outter = np.loadtxt(os.path.join(SAVE_FOLDER_OUTER + 'seg/' + f"{label}.eseg"))
    sseg_outter = np.loadtxt(os.path.join(SAVE_FOLDER_OUTER + 'sseg/' + f"{label}.seseg"))
        
    seg = np.concatenate([seg_inner, seg_outter])
    sseg = np.concatenate([sseg_inner, sseg_outter])
    print(i, c1 + c2, seg.mean() - 1)

    np.savetxt(os.path.join(SAVE_FOLDER_INNER_OUTER, f'seg/{label}.eseg'), seg)
    np.savetxt(os.path.join(SAVE_FOLDER_INNER_OUTER, f'sseg/{label}.seseg'), sseg)

/hdd/ibulygin/miniconda3/envs/fcd_hpc2/lib/python3.6/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


  0%|          | 0/27 [00:00<?, ?it/s]

26 6287289 0.0009223371154085669
26 6287289 0.0009223371154085669
26 6287289 0.0009223371154085669
26 6287289 0.0009223371154085669
26 6287289 0.0009223371154085669
26 6287289 0.0009223371154085669
26 6287289 0.0009223371154085669
26 6287289 0.0009223371154085669
26 6287289 0.0009223371154085669
26 6287289 0.0009223371154085669
26 6287289 0.0009223371154085669
26 6287289 0.0009223371154085669
26 6287289 0.0009223371154085669
26 6287289 0.0009223371154085669
26 6287289 0.0009223371154085669
26 6287289 0.0009223371154085669


## Preparing data for training MeshCNN

In [106]:
SAVE_FOLDER_INNER_OUTER

'../fcd_newdataset_meshes/inner_and_outer/'

In [107]:
ls -la ../fcd_newdataset_meshes/inner_and_outer/

total 8071128
drwxr-xr-x 4 ibulygin users      4096 сен 16 20:42 ./
drwxr-xr-x 6 ibulygin users      4096 сен 16 22:54 ../
-rw-r--r-- 1 ibulygin users 306100626 сен 16 19:43 0.1.obj
-rw-r--r-- 1 ibulygin users 306100626 сен 16 20:06 10.1.obj
-rw-r--r-- 1 ibulygin users 306100626 сен 16 20:08 11.1.obj
-rw-r--r-- 1 ibulygin users 306100626 сен 16 19:45 1.1.obj
-rw-r--r-- 1 ibulygin users 306100626 сен 16 20:10 12.1.obj
-rw-r--r-- 1 ibulygin users 306100626 сен 16 20:12 13.1.obj
-rw-r--r-- 1 ibulygin users 306100626 сен 16 20:15 14.1.obj
-rw-r--r-- 1 ibulygin users 306100626 сен 16 20:17 15.1.obj
-rw-r--r-- 1 ibulygin users 306100626 сен 16 20:19 16.1.obj
-rw-r--r-- 1 ibulygin users 306100626 сен 16 20:22 17.1.obj
-rw-r--r-- 1 ibulygin users 306100626 сен 16 20:24 18.1.obj
-rw-r--r-- 1 ibulygin users 306100626 сен 16 20:26 19.1.obj
-rw-r--r-- 1 ibulygin users 306100626 сен 16 20:28 20.1.obj
-rw-r--r-- 1 ibulygin users 306100626 сен 16 20:31 21.1.obj
-rw-r--r-- 1 ibulygin users 306100626 с

In [114]:
import shutil

FCD_FOLDER = '../fcd_newdataset_meshes/prepared/'

os.makedirs(os.path.join(FCD_FOLDER, 'train'), exist_ok=True)
os.makedirs(os.path.join(FCD_FOLDER, 'test'), exist_ok=True)
os.makedirs(os.path.join(FCD_FOLDER, 'seg'), exist_ok=True)
os.makedirs(os.path.join(FCD_FOLDER, 'sseg'), exist_ok=True)


if os.path.exists(FCD_FOLDER + 'test/cache'):
    shutil.rmtree(FCD_FOLDER + 'test/cache')
if os.path.exists(FCD_FOLDER + 'train/cache'):
    shutil.rmtree(FCD_FOLDER + 'train/cache')
    
files = glob.glob(FCD_FOLDER + 'test/*')
for f in files:
    os.remove(f)
files = glob.glob(FCD_FOLDER + 'train/*')
for f in files:
    os.remove(f)

test_idxs = ['0.1', '5.1', '15.1']
train_idxs = [label for label in brain_img_names.keys() if label not in test_idxs]

for label in train_idxs:
    shutil.copyfile(SAVE_FOLDER_INNER_OUTER + f"{label}.obj", FCD_FOLDER + 'train/' + f"{label}.obj")
    shutil.copyfile(SAVE_FOLDER_INNER_OUTER + 'seg/'+ f"{label}.eseg", FCD_FOLDER + 'seg/' + f"{label}.eseg")
    shutil.copyfile(SAVE_FOLDER_INNER_OUTER + 'sseg/'+ f"{label}.seseg", FCD_FOLDER + 'sseg/' + f"{label}.seseg")

for label in test_idxs:
    shutil.copyfile(SAVE_FOLDER_INNER_OUTER + f"{label}.obj", FCD_FOLDER + 'test/' + f"{label}.obj")
    shutil.copyfile(SAVE_FOLDER_INNER_OUTER + 'seg/'+ f"{label}.eseg", FCD_FOLDER + 'seg/' + f"{label}.eseg")
    shutil.copyfile(SAVE_FOLDER_INNER_OUTER + 'sseg/'+ f"{label}.seseg", FCD_FOLDER + 'sseg/' + f"{label}.seseg")   

## Shifting meshes from FreeSurfer

In [22]:
# N = 15
# base_names = ['lh.pial', 'lh.orig', 'rh.pial', 'rh.orig']
# def shift_mesh(i, base_name):
#     coords, faces, meta = nibabel.freesurfer.io.read_geometry(f'{i}_1_surf/{base_name}', read_metadata=True)
#     coords += meta['cras']
#     m = pymeshlab.Mesh(coords, faces)
#     ms = pymeshlab.MeshSet()
#     ms.add_mesh(m, base_name)
#     ms.save_current_mesh(f"meshlab_objects/{i}_{base_name}.obj")

# for i in range(N):
#     for base_name in base_names:
#         shift_mesh(i, base_name)

## Checking good value for gauss sigma and threshold value

In [21]:

# for i, br_name in enumerate(brain_img_names):
#     print(i)
#     brain = nibabel.load(br_name)
#     mask = nibabel.load(brain_mask_names[i]).get_fdata() > 0
#     brain_tensor = brain.get_fdata() > 95
#     l = 7
#     brain_tensor = gaussian_filter(brain_tensor.astype('float'), sigma=l)
#     nifti = nibabel.Nifti1Image(brain_tensor.astype('float'), brain.affine)
#     nibabel.save(nifti, f"{i}_gauss_g_{l}.nii.gz")
#     t = 0.4
#     nifti_t = nibabel.Nifti1Image((brain_tensor > t).astype('float'), brain.affine)
#     nibabel.save(nifti_t, f"{i}_gauss_g_{l}_t_{t}.nii.gz") 